In [42]:
import json
import pandas as pd
from pathlib import Path
from utils import clean_extracted_data, compute_segmentation_scores

In [44]:
# Load extracted data
output_dir = Path('../../outputs/nv_task/extractions')

subset = [
    '*NOMC-zeroshot_NuExtract-v1.5.jsonl',
    '*NOMC-noexample-zeroshot_NuExtract-2-4B_temperature=0_do_sample=False.jsonl',
    '*gpt-4o-mini-2024-07-18*'
    ]


# Extract substring after NuExtract for each output
nv_task_nu_json = {}
for s in subset:
    t = list(output_dir.glob(s))[0]

    # Everying after zeroshot_
    key = t.stem.split('zeroshot_')[1]
    with open(t, 'r') as f:
        nv_task_nu_json[key] = json.load(f)

nv_task_nu_dict = {}
for params, data in nv_task_nu_json.items():
    nv_task_nu_dict[params] = clean_extracted_data(data)

In [45]:
# Load annotations

from labelrepo.projects.nv_task import load_annotations
from utils import get_annotation_summary, run_all_comparisons

annotations = load_annotations()
annotations = annotations[annotations['annotator_name'] != 'alice_chen']

annotations_summary = get_annotation_summary(annotations)

In [97]:
all_scores_df = {}
for params, data in nv_task_nu_dict.items():
    print(f"Running comparisons for {params}")
    scores_df = run_all_comparisons(annotations_summary, data)
    all_scores_df[params] = scores_df

Running comparisons for NuExtract-v1.5
Running comparisons for NuExtract-2-4B_temperature=0_do_sample=False
Running comparisons for gpt-4o-mini-2024-07-18


In [47]:
exclude_idx = [_p for _p, v in annotations_summary.items() if v['Exclude'] is not None]
has_task_name = set([_p for _p, v in annotations_summary.items() if v['TaskName'] and ('n/a' not in v['TaskName'])]) - set(exclude_idx)
has_task_noname = set([_p for _p, v in annotations_summary.items() if not(v['TaskName']) or 'n/a' in v['TaskName']]) - set(exclude_idx)

In [48]:
results = {}
nas_per_label = {}
for params, df in all_scores_df.items():
    results[params], nas_per_label[params] = compute_segmentation_scores(df, exclude_idx, has_task_name, has_task_noname)

In [49]:
# Sort by key
results = dict(sorted(results.items()))
for params, df in results.items():
    print(f"Results for {params}")
    display(df)
    display(nas_per_label[params])
    print("\n")

Results for NuExtract-2-4B_temperature=0_do_sample=False


,n,HasRestingState,TaskName,TaskDescription
All,103,0.13,0.68,0.81
No Exclude,94,0.14,0.68,0.81
Has Task Name,63,0.05,0.81,0.82
Has Task with No Name,31,0.32,0.22,0.78


,HasRestingState,TaskName,TaskDescription
All,0.0,0.21,0.37
No Exclude,0.0,0.14,0.31
Has Task Name,0.0,0.00,0.14
Has Task with No Name,0.0,0.42,0.65




Results for NuExtract-v1.5


,n,HasRestingState,TaskName,TaskDescription
All,104,0.90,0.80,0.73
No Exclude,95,0.89,0.80,0.73
Has Task Name,64,0.97,0.91,0.71
Has Task with No Name,31,0.74,0.20,0.86


,HasRestingState,TaskName,TaskDescription
All,0.0,0.38,0.50
No Exclude,0.0,0.32,0.45
Has Task Name,0.0,0.14,0.28
Has Task with No Name,0.0,0.68,0.81




Results for gpt-4o-mini-2024-07-18


,n,HasRestingState,Modality,Exclude,TaskName,TaskDescription
All,104,0.91,0.99,0.0,0.78,0.77
No Exclude,95,0.91,0.99,NaN,0.78,0.77
Has Task Name,64,0.97,1.00,NaN,0.94,0.77
Has Task with No Name,31,0.77,0.97,NaN,0.18,0.77


,HasRestingState,Modality,Exclude,TaskName,TaskDescription
All,0.0,0.09,0.97,0.21,0.37
No Exclude,0.0,0.00,1.00,0.14,0.31
Has Task Name,0.0,0.00,1.00,0.00,0.14
Has Task with No Name,0.0,0.00,1.00,0.42,0.65


## Manual comparison

In [100]:
# Combine all results

combined_scores = []
for annotator_name, _df in all_scores_df.items():
    _df = _df.copy()
    _df.columns = [f"{c}_score" for c in _df.columns]
    _df = _df.reset_index().rename(columns={'index': 'pmcid'})
    _df['annotator_name'] = annotator_name
    combined_scores.append(_df)

combined_scores = pd.concat(combined_scores)


combined = []
for params, df in nv_task_nu_dict.items():
    nu = pd.DataFrame(df).T
    nu['annotator_name'] = f'{params}'
    if 'id' in nu:
        nu['pmcid'] = nu['id']
    combined.append(nu)

combined.append(pd.DataFrame(annotations_summary).T)

combined_df = pd.concat(combined)

combined_df = combined_df.merge(combined_scores, on=['pmcid', 'annotator_name'], how='left')

In [118]:
import pandas as pd
from IPython.display import display

pd.set_option('max_colwidth', 300)
    

def _display(df, pmcids=None, cols = ['annotator_name', 'StudyObjective', 'TaskDescription', 'TaskName', 'TaskDescription_score', 'TaskName_score'], sort_by='TaskName_score'):
    if pmcids:
        df = df[df.pmcid.isin(pmcids)]

    # PMCIDS sorted by mean score
    pmcids = df.groupby('pmcid')[sort_by].mean().sort_values(ascending=True).index

    for pmcid in pmcids:
        display(df[df.pmcid == pmcid][cols])

In [119]:
combined_has_task = combined_df[combined_df.pmcid.isin(has_task_name)]
combined_no_task = combined_df[combined_df.pmcid.isin(has_task_noname)]

In [121]:
_display(combined_has_task, sort_by='TaskDescription_score')

,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
81,NuExtract-v1.5,Probing the relevance of the hippocampus for conflict-induced memory improvement,[during functional magnetic resonance imaging (fMRI)],[face-word Stroop task],0.11,1.0
186,NuExtract-2-4B_temperature=0_do_sample=False,Probing the relevance of the hippocampus for conflict-induced memory improvement,"[Participants performed a face-word Stroop task during functional magnetic resonance imaging (fMRI) followed by a recognition task for the faces., About 30–45 min after the face-word Stroop task, an incidental memory task of to the previously seen faces was conducted outside of the MRI scanner.]","[face-word Stroop task, incidental delayed memory task]",0.54,1.0
290,gpt-4o-mini-2024-07-18,To investigate how hippocampal lesions in MTLE patients affect the impact of conflict resolution on memory encoding at a behavioral level and at the level of BOLD responses.,[Participants performed a face-word Stroop task during functional magnetic resonance imaging (fMRI).],[Face-word Stroop task],0.3,1.0
378,delavega_nv,NaN,"[The task comprised face-word stimuli consisting of male and female faces [Glasgow Face Database, ], overlaid by the words “man” or “woman” (in German language). These words could be either congruent or incongruent with the sex of the face ( A). The paradigm consisted of the 120 faces (60 males...",[Face-word Stroop task],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
6,NuExtract-v1.5,to test the hypothesis that there are dissociable brain networks related to the expectancy and associative strength theories using a single-cue fear conditioning paradigm with a pseudo-random intermittent reinforcement schedule during functional magnetic resonance imaging,[a single-cue fear conditioning paradigm with a pseudo-random intermittent reinforcement schedule],[single-cue fear conditioning],0.28,0.9
112,NuExtract-2-4B_temperature=0_do_sample=False,The neurocognitive processes underlying Pavlovian conditioning in humans are still largely debated.,"[A repeated-measures ANOVA on expectancy ratings performed with trial sequence (6 levels) as a within-subject variable revealed a significant main effect of trial sequence [ F (5, 100) = 57.98, P < 0.001, = 0.744]. A significant linear trend was also identified, indicating that expectanc...","[Expectancy ratings, fMRI univariate whole-brain results, fMRI univariate ROI results, FMRI exploratory FIR results]",0.55,0.4
217,gpt-4o-mini-2024-07-18,The present study tested the hypothesis that there are dissociable brain networks related to the expectancy and associative strength theories using a single-cue fear conditioning paradigm with a pseudo-random intermittent reinforcement schedule during functional magnetic resonance imaging.,[This task involves the presentation of a conditioned stimulus (CS) followed by an unconditioned stimulus (US) to elicit conditioned responses (CRs).],[Single-Cue Fear Conditioning],0.31,0.9
409,delavega-aliceoverlap,NaN,[The experimental design was based on the one used by and . The main task consisted of three runs (i.e. functional runs in the scanner) of 45 trials per run for a total of 135 trials across all three runs. Half the trials were CS-alone trials in which the US was omitted. In the other half of ...,[single-cue conditioning paradigm],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
49,NuExtract-v1.5,examined the spatiotemporal dynamics of the neural activity underlying negative and neutral involuntary memory retrieval,[while undergoing a functional magnetic resonance imaging scan],[involuntary memory task],0.18,1.0
155,NuExtract-2-4B_temperature=0_do_sample=False,Neural responses to emotional involuntary memories in posttraumatic stress disorder: Differences in timing and activity,"[Twenty-one individuals with PTSD and 21 non-PTSD, trauma-exposed controls performed an involuntary memory task, while undergoing a functional magnetic resonance imaging scan. Environmental sounds served as cues for well-associated pictures of negative and neutral scenes. We used a finite impuls...","[Involuntary memory task, Post-scan recall]",0.44,1.0
258,gpt-4o-mini-2024-07-18,To examine the spatiotemporal dynamics of the neural activity underlying negative and neutral involuntary memory retrieval in individuals with PTSD compared to trauma-exposed controls.,"[Participants performed an involuntary memory task while undergoing fMRI scanning, using environmental sounds as cues for associated pictures.]",[Involuntary Memory Task],0.55,1.0
346,delavega-other,NaN,"[During the re-encoding runs, each sound-picture pair and unpaired sound was presented. Participants rated on a 7-point scale how emotional the stimuli were. During the recall runs, the 55 paired and 40 unpaired sounds (randomly intermixed) were presented, panned 15° to either the left or the ri...",[involuntary memory],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
63,NuExtract-v1.5,to investigate the relationship between food cue reactivity in the striatum measured one month after the onset of a weight loss program and weight changes obtained by the end of the six-month intervention,"[one before starting the six-month weight loss intervention (baseline, Session 1 ) and another at the end of the first month of the six-month weight loss intervention (Session 2 )]",[fMRI imaging sessions],0.26,0.25
169,NuExtract-2-4B_temperature=0_do_sample=False,The efficacy of weight loss intervention can be predicted based on early alterations of fMRI food cue reactivity in the striatum.,"[The study aimed to investigate the relationship between food cue reactivity in the striatum measured one month after the onset of the weight loss program and weight changes obtained at the end of the six-month intervention., The study aimed to investigate the relationship between food cue react...","[fMRI food cue reactivity, fMRI food cue reactivity]",0.28,1.0
272,gpt-4o-mini-2024-07-18,To investigate the relationship between food cue reactivity in the striatum measured one month after the onset of the weight loss program and weight changes obtained at the end of the six-month intervention.,"[Participants viewed images of neutral objects, high-calorie foods, and low-calorie foods to measure brain activation in response to food cues.]",[Food Cue Reactivity Task],0.81,1.0
360,delavega_nv,NaN,"[During the experimental runs images of neutral objects, high-calorie foods, and low-calorie foods were presented in a block design format. Each run consisted of four randomly presented 21 s long blocks for each image category in which 7 individual images were presented for 2 s followed by a 1 s...",[food cue reactivity],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
55,NuExtract-v1.5,to replicate and extend previous findings on the psychological and neural mechanisms underlying emotion explosiveness and accumulation,[],[],<NA>,<NA>
161,NuExtract-2-4B_temperature=0_do_sample=False,The impact of self-distancing on emotion explosiveness and accumulation: An fMRI study,"[Participants were asked to adopt a self-immersed or self-distanced perspective while reading and thinking about negative social feedback, and to report on felt changes in negative affect during that period using an emotion intensity profile tracking approach., Participants were asked to adopt a...","[Emotion intensity profile tracking, Emotion intensity profile tracking]",0.47,0.31
264,gpt-4o-mini-2024-07-18,"To replicate and extend previous findings on the psychological and neural mechanisms underlying emotion explosiveness and accumulation, specifically examining the effects of self-distancing versus self-immersion on these emotional dynamics.",[Participants were asked to adopt a self-immersed or self-distanced perspective while reading and thinking about negative social feedback.],[Self-Distancing vs. Self-Immersion Perspective Task],0.52,0.32
352,delavega_nv,NaN,"[Following previous studies [ – ], negative social feedback was used to induce emotions for two reasons: (a) in daily life emotions are often caused by social stimuli [ , ] and (b) social feedback elicits emotional responses that are long enough to study emotion dynamics [ ]. The social feedback...",[Social feedback paradigm],NaN,NaN


...

,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
65,NuExtract-v1.5,to show that the cerebellum has to be added to the neural network processing predictions errors in learned fear responses,[],[],<NA>,<NA>
171,NuExtract-2-4B_temperature=0_do_sample=False,The cerebellum is involved in processing of predictions and prediction errors in a fear conditioning paradigm,"[All MR images were acquired with the participants lying supine inside a whole-body MRI system operating at 7 Tesla magnetic field strength (MAGNETOM 7T, Siemens Healthcare GmbH, Erlangen, Germany) equipped with a one-channel transmit/32 channel receive RF head coil (Nova Medical, Wilmington, MA...","[fMRI acquisition, fMRI analysis]",<NA>,0.42
274,gpt-4o-mini-2024-07-18,To address the contribution of the cerebellum in processing predictions and prediction errors during a fear conditioning paradigm.,[Participants underwent a differential fear conditioning paradigm during fMRI to study cerebellar activation related to predictions and prediction errors.],[Fear Conditioning Paradigm],<NA>,1.0
362,delavega_nv,NaN,None,[fear conditioning and extinction paradigm],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
67,NuExtract-v1.5,probe neural responsiveness to anticipated and received rewards in the context of bipolar disorder,[scanned individuals with remitted bipolar I disorder ( n = 24) and well-matched controls ( n = 24; matched for age and gender)],[Monetary Incentive Delay (MID) task],<NA>,1.0
173,NuExtract-2-4B_temperature=0_do_sample=False,To examine neural responses to monetary incentives in individuals diagnosed with Bipolar Disorder.,[Participants completed a version of the Monetary Incentive Delay (MID) Task designed to elicit neural and behavioral responses to monetary incentives and their outcomes during FMRI scan acquisition.],[Monetary Incentive Delay (MID) Task],<NA>,1.0
276,gpt-4o-mini-2024-07-18,To probe neural responsiveness to anticipated and received rewards in the context of bipolar disorder.,[Designed to elicit neural and behavioral responses to monetary incentives and their outcomes during FMRI scan acquisition.],[Monetary Incentive Delay (MID) Task],<NA>,1.0
364,delavega_nv,NaN,None,[Monetary Incentive Delay (MID) Task],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
72,NuExtract-v1.5,to examine the relationship between sleep disruption–induced pain sensitivity and functional connectivity (FC) of cognitive networks contributing to pain modulation,[designed to elicit intrinsic network connectivity without the risk of participants falling asleep],[simple motor task],<NA>,1.0
178,NuExtract-2-4B_temperature=0_do_sample=False,Individual differences in pain sensitivity are associated with cognitive network functional connectivity following one night of experimental sleep disruption,"[Participants were given the opportunity for 8 hours of sleep at the CRU without experimental disruption., Participants completed a quantitative sensory testing session before scanning at both study visits to derive participant‐tailored temperatures for the fMRI noxious stimulation task.]","[Simple motor task, Noxious thermal stimulation task]",<NA>,1.0
281,gpt-4o-mini-2024-07-18,To examine the relationship between sleep disruption-induced pain sensitivity and functional connectivity (FC) of cognitive networks contributing to pain modulation.,[A task designed to elicit intrinsic network connectivity without the risk of participants falling asleep.],[Simple Motor Task (SMT)],<NA>,1.0
369,delavega_nv,NaN,None,[simple motor task (SMT)],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
96,NuExtract-v1.5,Understanding the mechanisms underlying aging-related memory decline is likely to inform the development of compensatory strategies promoting independence in old age.,[an adaptation of a well-established paradigm],[spatial contextual memory task],<NA>,1.0
201,NuExtract-2-4B_temperature=0_do_sample=False,Decreased Efficiency of Between-Network Dynamics During Early Memory Consolidation With Aging,"[Participants had to memorize the object and its respective position., The consolidation phase was interrupted by a modulatory task that differed between conditions in the extent to which consolidation processes were supposed to be affected., During the retrieval task, the stimuli of the encodin...","[encoding, consolidation, retrieval]",<NA>,0.33
305,gpt-4o-mini-2024-07-18,This study examined age effects on consolidation-related neural mechanisms and their susceptibility to interference using functional magnetic resonance imaging data from younger and older healthy participants.,"[Participants performed a spatial contextual memory task consisting of an encoding, consolidation, and retrieval phase.]",[Spatial Contextual Memory Task],<NA>,1.0
393,delavega_nv,NaN,None,[spatial contextual memory task],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
104,NuExtract-v1.5,to investigate behavioral and neural associations in self-regulation across the two domains in human participants,"[reappraisal of positive and negative emotional stimuli, dietary self-control task]","[emotion reappraisal, dietary self-control]",<NA>,1.0
209,NuExtract-2-4B_temperature=0_do_sample=False,We combined established emotion regulation and dietary choice tasks with fMRI to investigate behavioral and neural associations in self-regulation across the two domains in human participants.,"[Participants were asked to either (1) simply view and react naturally or (2) reappraise photographs with different emotional valence., Participants knew that one of these trials would be selected at random and their choice on that trial implemented for real, meaning that they would have to eat ...","[emotion reappraisal, food choice]",<NA>,0.695
313,gpt-4o-mini-2024-07-18,To investigate behavioral and neural associations in self-regulation across emotion regulation and dietary choice tasks in human participants.,"[Participants were presented with positive, negative, and neutral stimuli and were asked to either view or reappraise the content to regulate their feelings., Participants made choices about whether to eat presented food items based on their healthiness and tastiness ratings.]","[Emotion Regulation Task, Dietary Self-Control Task]",<NA>,0.81
401,delavega_nv,NaN,None,"[emotion reappraisal, dietary self-control]",NaN,NaN


In [122]:
_display(combined_no_task, sort_by='TaskDescription_score')

,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
34,NuExtract-v1.5,to investigate how the two factors (giver and type of gift) interactively determine the neural reward value of the gift,"[Imagine you are having a birthday party at home. You have two male friends attending the party, one whom you prefer, and the other whom you are indifferent to. Each of them talks to you privately. Each tells you that he brought a birthday gift especially for you. Please judge the attractiveness...",[],0.46,<NA>
140,NuExtract-2-4B_temperature=0_do_sample=False,To investigate how the neural processing for understanding a gift’s social meaning is modulated by preferences for the giver.,"[Participants were asked to judge the value of gifts from two male acquaintances they knew in real life: for each female participant, one male giver was preferred, while she was indifferent toward the other giver (i.e., the “liked” and “uninteresting” givers). The social meaning of the gift (i.e...",[Gift Judgement Task],0.54,0.18
244,gpt-4o-mini-2024-07-18,"To investigate how preferences for the giver and the type of gift interactively determine the reward value of a gift, specifically examining the neural processing for understanding a gift's social meaning.","[Participants judged the attractiveness of gifts received from male acquaintances, focusing on the gift rather than the giver.]",[Gift Attractiveness Judgment Task],0.72,0.17
332,delavega_nv,NaN,[A) The task was conducted in an event-related design. Six trials from either the liked or the uninteresting male acquaintance were combined in a set. Each set contained pseudo-randomized four target trials (shown in red) with two control trials (shown in blue). (B) Each trial was created by a 2...,[n/a],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
42,NuExtract-v1.5,to study the neural mechanisms underlying the disambiguation of degraded spoken sentences in situations where the subject has previously been exposed to the intact versions of the sentences,[],[],<NA>,<NA>
148,NuExtract-2-4B_temperature=0_do_sample=False,Predictive processing increases intelligibility of acoustically distorted speech: Behavioral and neural correlates,"[The fMRI experiment was divided into two 19‐min functional runs and one 6‐min anatomical run at the end of the scanning session. Each functional run consisted of 9 D‐I‐D stimulus sets, each of which comprised three blocks of six sentences (see Figure ). The blocks were 22 or 24 s in duration. ...","[fMRI experiment, Behavioral experiment]",0.795,0.25
251,gpt-4o-mini-2024-07-18,We examined which brain areas are involved in the comprehension of acoustically distorted speech using an experimental paradigm where the same distorted sentence can be perceived at different levels of intelligibility.,"[The task involved presenting distorted sentences followed by their intact counterparts, and then the distorted sentences again to assess changes in intelligibility.]",[DID stimulus set],0.61,0.11
339,delavega-aliceoverlap,NaN,"[a degraded sentence is first experienced as very difficult to understand and, after a single presentation of its intact counterpart, the intelligibility of this same degraded sentence reaches near‐perfect level., functional run consisted of 9 D‐I‐D stimulus sets, each of which comprised three b...",[n/a],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
61,NuExtract-v1.5,to investigate whether individual differences in recruitment of cognitive control regions during a difficult response inhibition task are associated with a failure to regulate neural responses to rewarding food cues in a subsequent task in a cohort of 27 female dieters,"[participants were required to actively inhibit reading a series of words that appeared on the screen over the course of seven minutes, involving food commercials that has previously been shown to reliably recruit the brain’s reward system]","[effortful self-control task, food-cue reactivity task]",0.73,0.13
167,NuExtract-2-4B_temperature=0_do_sample=False,Investigate whether individual differences in recruitment of cognitive control regions during a difficult response inhibition task are associated with a failure to regulate neural responses to rewarding food cues in a subsequent task in a cohort of 27 female dieters.,"[Participants were required to actively inhibit reading a series of words that appeared on the screen over the course of seven minutes., Participants engaged in a food-cue reactivity task involving food commercials that has previously been shown to reliably recruit the brain’s reward system.]","[Effortful self-control task, Food cue reactivity task]",0.73,0.13
270,gpt-4o-mini-2024-07-18,Investigate whether individual differences in recruitment of cognitive control regions during a difficult response inhibition task are associated with a failure to regulate neural responses to rewarding food cues in a subsequent task in a cohort of 27 female dieters.,[Participants were required to actively inhibit reading a series of words that appeared on the screen while watching a nature documentary.],[Effortful Self-Control Task],0.73,0.12
358,delavega_nv,NaN,"[For this task, all participants were told that they would watch a clip from a nature documentary featuring Canadian bighorn sheep, and at times various words would appear and move around on the screen, but they were told to “avoid reading these distractor words whenever they appear.” On average...","[n/a, watched]",NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
84,NuExtract-v1.5,"Testing the degree with which healthcare providers take into account feedbacks about their pain assessment, and whether they weight in different extent the opinion of the patient from that of other physicians.",[],[],<NA>,<NA>
189,NuExtract-2-4B_temperature=0_do_sample=False,Medical education and distrust modulate the response of insular-cingulate network and ventral striatum in pain diagnosis,"[Participants rated the pain of facial expressions and were confronted with two feedbacks (self-report of the person in pain and average opinion of 20 medical practitioners)., Participants were asked to rate again all 44 videoclips without being exposed to any feedback.]","[Pain expression ratings, Feedback processing]",0.79,0.27
293,gpt-4o-mini-2024-07-18,To test whether senior medical students differed from younger colleagues and lay controls in the way they assess people's pain and take into consideration their feedback.,[Participants appraised the pain of facial expression video-clips and subsequently were confronted with feedbacks from the person in pain and the average opinion of medical practitioners.],[Pain Assessment Task],0.67,0.26
381,delavega_nv,NaN,"[The main experimental session was organized in three blocks. In the first block, participants saw a fixation cross (1 s) followed by one video (3 s) and were asked to estimate the amount of pain experienced by the person in the video, by moving a randomly-presented rectangular cursor on a Visua...",[n/a],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
60,NuExtract-v1.5,"Here, we examine social information processing in children with ASD by probing brain circuit function and connectivity in response to human vocal sounds.","[presented in 10 separate runs, each lasting 4 min]",[Auditory stimuli],1.0,0.21
166,NuExtract-2-4B_temperature=0_do_sample=False,Impaired voice processing in reward and salience circuits predicts social communication in children with autism,"[Children were presented with unfamiliar voices and environmental sounds during fMRI scanning. The task was to identify their mother's voice with high accuracy., Children were presented with mother's voice and unfamiliar voices during fMRI scanning. The task was to identify their mother's voice ...","[Unfamiliar Voices minus Environmental Sounds, Mother’s Voice minus Unfamiliar Voices]",0.84,0.13
269,gpt-4o-mini-2024-07-18,To examine social information processing in children with autism spectrum disorder (ASD) by probing brain circuit function and connectivity in response to human vocal sounds.,[The task involved processing unfamiliar voices and mothers' voices to assess brain activity and connectivity.],[Voice Processing Task],0.45,0.17
357,delavega_nv,NaN,"[Auditory stimuli were presented in 10 separate runs, each lasting 4 min. One run consisted of 56 trials of mother’s voice, unfamiliar female voices, environmental sounds and catch trials, which were pseudo-randomly ordered within each run. Stimulus presentation order was the same for each subje...",[n/a],NaN,NaN


...

,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
95,NuExtract-v1.5,To investigate whether increased hemodynamic activity in the PPA that is usually detected by contrasting blocks of pictures is also present under more natural conditions.,"[offering ecologically more valid visual stimulation than a paradigm using blocks of pictures, offering an exclusively auditory stimulation]","[audio-visual movie, audio-description]",<NA>,0.3
200,NuExtract-2-4B_temperature=0_do_sample=False,Processing of visual and non-visual naturalistic spatial information in the 'parahippocampal place area',"[The movie stimulus offers ecologically more valid visual stimulation than a paradigm using blocks of pictures., The audio-description stimulus offers an exclusively auditory stimulation.]","[movie stimulus, audio-description stimulus]",<NA>,0.21
304,gpt-4o-mini-2024-07-18,"To investigate the perception of scene-related, spatial information embedded in two naturalistic stimuli (a movie and its audio-description) and to model hemodynamic activity based on annotations of selected stimulus features.",[Participants watched a Hollywood movie and listened to its audio-description to investigate spatial information processing.],[Naturalistic Stimuli Presentation],<NA>,0.17
392,delavega_nv,NaN,None,[n/a],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
98,NuExtract-v1.5,"evaluate correlation of sleep quality with visual attention and search, functional, and tracts’ properties of the DAN and VAN",[visual search and attention],[computerized match-to-sample (MTS) task],<NA>,<NA>
203,NuExtract-2-4B_temperature=0_do_sample=False,"The aim of the study was to evaluate correlation of sleep quality with visual attention and search, functional, and tracts’ properties of the DAN and VAN.","[In this computerized task, subjects were presented with a complex figure in the middle of the screen. Then, a few patterns were shown in the periphery, from which one was matched with the presented pattern. In the first trials, two patterns were presented in the periphery, and it was increased ...",[match-to-sample (MTS) task],<NA>,<NA>
307,gpt-4o-mini-2024-07-18,"The aim of the study was to evaluate correlation of sleep quality with visual attention and search, functional, and tracts properties of the DAN and VAN.",[This computerized task assesses visual search and attention by presenting a complex figure and requiring subjects to match it with patterns shown in the periphery.],[match-to-sample (MTS) task],<NA>,<NA>
395,delavega-aliceoverlap,NaN,None,[],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
102,NuExtract-v1.5,re-examined the VAN and the DAN neuroanatomy by co-registering individual network maps in a common functional space,[],[],<NA>,<NA>
207,NuExtract-2-4B_temperature=0_do_sample=False,Re-examined the VAN and the DAN neuroanatomy by co-registering individual network maps in a common functional space.,[Used 110 7 T resting-state functional MRI datasets from the Human Connectome Project S1200.],[Resting-state functional imaging (rs-fMRI)],<NA>,<NA>
311,gpt-4o-mini-2024-07-18,To explore the subcortical anatomy of the ventral and dorsal attention networks by aligning individual resting-state functional maps in a common functional space.,[],[],<NA>,<NA>
399,delavega_nv,NaN,None,[],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
0,NuExtract-v1.5,Understanding how the human brain gives rise to consciousness remains a grand challenge for modern neuroscience.,[measured behavioural responsiveness],[target detection task],<NA>,0.17
106,NuExtract-2-4B_temperature=0_do_sample=False,"To ask whether the connectivity within and between three networks that are primary sites of propofol‐induced sedation and key to conscious cognition—the DAN, ECN, and DMN—underlie individual responsiveness differences under anaesthesia.","[Participants were asked to perform a computerized auditory target detection task (50 trials), which aimed to assess individual responsiveness differences during moderate anaesthesia., Participants underwent two functional scans during wakefulness and moderate anaesthesia. A plot‐driven auditory...","[Auditory target detection task, Resting‐state fMRI]",<NA>,0.19
211,gpt-4o-mini-2024-07-18,"To investigate whether variability or impairments in functional connectivity within and between the dorsal attention network (DAN), executive control network (ECN), and default mode network (DMN) underlie individual differences in responsiveness during propofol anaesthesia.",[A computerized auditory target detection task aimed to assess individual responsiveness differences during moderate anaesthesia.],[Auditory Target Detection Task],<NA>,0.18
403,delavega_nv,NaN,None,[n/a],NaN,NaN


,annotator_name,StudyObjective,TaskDescription,TaskName,TaskDescription_score,TaskName_score
1,NuExtract-v1.5,"To develop specific treatments for FTD, we need to understand the underlying pathobiology.",[],[],<NA>,<NA>
107,NuExtract-2-4B_temperature=0_do_sample=False,To explore the neural correlates of four distinct dimensions of Formal Thought Disorder (FTD) using perfusion and structural MRI.,[],[],<NA>,<NA>
212,gpt-4o-mini-2024-07-18,This study tested whether four FTD dimensions differ in their association with brain perfusion and brain structure.,[],[],<NA>,<NA>
404,delavega_nv,NaN,None,[],NaN,NaN


It's difficult to judge objectively, but at first glance, GPT's descritions are more useful than the ones from NuExtract. NE 1.5 often gives fragments which are not useful by themselves. While NE 2 is more likely to just be incorrect. The higher scores of NuExtract likely stem from the fact that its more likely to quote verbatim, but the actual text it quotes is not necessaril the most usefu.  